
<h2> import </h2>


In [1]:
import geemap
import os
import ee
import matplotlib.pyplot as plt
from geemap import cartoee
import cartopy.crs as ccrs
from sqlalchemy import false, true

geemap.ee_initialize()

C:\Users\alalala\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<h3>設定年份經緯度</h3>

In [8]:
#經緯度要到小數點後面一點 
lat = 133.35 # 緯度
lon =  -31.372# 經度
square = 0.8

#設定範圍
# yMin = lat - square
# xMin = lon - square
# yMax = lat + square
# xMax = lon + square
# rectangle = ee.Geometry.Rectangle(
#   [xMin, yMin, xMax, yMax]
# )
point = ee.Geometry.Point(lon,lat)
#設定時間
start_year = 1984
end_year = 2012
years = ee.List.sequence(start_year, end_year)




<h3> 選取圖片 </h3>


In [9]:
def get_best_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    image = (
        ee.ImageCollection("LANDSAT/LT05/C02/T1_TOA")
        .filterBounds(point)#設定範圍
        .filter(ee.Filter.calendarRange(1, 12, 'month'))#指定月份 1-12月
        .filterDate(start_date, end_date)
        .sort('CLOUD_COVER').first()#去雲
        )
    return ee.Image(image)

#圖像可視化
#https://developers.google.com/earth-engine/guides/image_visualization
vis_params = {"bands": ['B3', 'B2', 'B1'],#rbg
              "min": 0.0,
              "max": 0.4,
              "gamma" : 1.2, #亮度             
            }

collection = ee.ImageCollection(years.map(get_best_image))
image = ee.Image(collection.first())

<h3>從地圖上看選取範圍<h3>


In [10]:
#因為跑要時間所以註掉
Map = geemap.Map()
Map.addLayer(image, vis_params, 'First image')
Map.setCenter(lon, lat, 8)
Map

Map(center=[133.35, -31.372], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…


<h3> 顯示圖片 </h3>


In [ ]:
#設定範圍
w = 0.25
h = 0.25
region = [lon - w, lat - h, lon + w, lat + h]

fig = plt.figure(figsize=(10, 7))
ax = cartoee.get_map(image, region=region, vis_params=vis_params)
plt.show()

<h3>下載圖片</h3>

In [ ]:
#collection.aggregate_array('system:index').getInfo()

In [ ]:
#設定路徑及資料夾名稱
downloads_name = str(int(lon)) +","+ str(int(lat))
downloads_path = "~/Downloads/" + downloads_name

#下載圖片 要用再解註
# cartoee.get_image_collection_gif(
#     ee_ic=collection,
#     out_dir=os.path.expanduser(downloads_path),
#     out_gif="1984-2012_timelapse.gif",
#     vis_params=vis_params,
#     region=region,#選取範圍
#     fps=3,
#     plot_title=" ",
#     date_format='YYYY-MM-dd',
#     fig_size=(10, 8),
#     dpi_plot=100,
#     file_format="png",
#     verbose=True,
#   )
